In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/covid19-case-surveillance-public-use-dataset/COVID-19_Case_Surveillance_Public_Use_Data.csv')
data

In [ ]:
missing=data.isnull().sum()
missing

In [ ]:
data_1=data.copy()
data_1

In [ ]:
#since there are more than 80% of missing in pos_spec_dt and onset_dt
#we are going to drop
data_1=data_1.drop(['onset_dt','pos_spec_dt'],axis=1)
data_1=data_1.dropna()
data_1

In [ ]:
missing_1=data_1.isnull().sum()
missing_1

In [ ]:
#to handle date time 
month=data_1['cdc_report_dt'].apply(lambda x:x.split('/')[1])
day=data_1['cdc_report_dt'].apply(lambda x:x.split('/')[2])
day

In [ ]:
data_2=data_1.copy()
data_2

In [ ]:
data_2['cdc_report_day']=day
data_2['cdc_report_month']=month
data_2=data_2.drop(['cdc_report_dt'],axis=1)
data_2

In [ ]:
unique= data_2.nunique()
unique

In [ ]:
columns_cat=data_2[['sex','age_group','Race and ethnicity (combined)','hosp_yn','icu_yn','death_yn','medcond_yn','current_status']]


In [ ]:
columns_cat

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
for i in columns_cat:
    sns.barplot(columns_cat[i].value_counts().index,columns_cat[i].value_counts()).set_title(i)
    plt.show()

In [ ]:
print(pd.pivot_table(data_2,index='current_status',columns='sex',aggfunc='count'))
print(pd.pivot_table(data_2,index='current_status',columns='age_group',aggfunc='count'))
print(pd.pivot_table(data_2,index='current_status',columns='Race and ethnicity (combined)',aggfunc='count'))
print(pd.pivot_table(data_2,index='current_status',columns='hosp_yn',aggfunc='count'))
print(pd.pivot_table(data_2,index='current_status',columns='icu_yn',aggfunc='count'))
print(pd.pivot_table(data_2,index='current_status',columns='death_yn',aggfunc='count'))

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [ ]:
data_3=data_2.copy()
data_3

In [ ]:
data_3['sex']= label_encoder.fit_transform(data_3['sex'])
data_3['age_group']= label_encoder.fit_transform(data_3['age_group'])
data_3['Race and ethnicity (combined)']= label_encoder.fit_transform(data_3['Race and ethnicity (combined)'])
data_3['hosp_yn']= label_encoder.fit_transform(data_3['hosp_yn'])
data_3['icu_yn']= label_encoder.fit_transform(data_3['icu_yn'])
data_3['death_yn']= label_encoder.fit_transform(data_3['death_yn'])
data_3['medcond_yn']= label_encoder.fit_transform(data_3['medcond_yn'])  
data_3                                                         

In [ ]:
data_3['current_status']= label_encoder.fit_transform(data_3['current_status'])

In [ ]:
X=data_3.drop(['current_status'],axis=1)
Y=data_3['current_status']
from sklearn.model_selection import train_test_split
X_Train,X_test,Y_Train,Y_Test=train_test_split(X,Y,test_size=0.2,random_state=2)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_Train,Y_Train)
prediction = model.predict(X_test)
print(prediction)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score=accuracy_score(Y_Test,prediction)
print(accuracy_score)

In [ ]:
submission=pd.DataFrame({"current_status":prediction})

In [ ]:
submission.to_csv('covid_19.csv')